In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [3]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [4]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"])
train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)

9913344it [00:00, 121594424.94it/s]        

Extracting ./tmp_files/MNIST/raw/train-images-idx3-ubyte.gz to ./tmp_files/MNIST/raw


29696it [00:00, 91786331.31it/s]         


Extracting ./tmp_files/MNIST/raw/train-labels-idx1-ubyte.gz to ./tmp_files/MNIST/raw



1649664it [00:00, 72139544.95it/s]         

Extracting ./tmp_files/MNIST/raw/t10k-images-idx3-ubyte.gz to ./tmp_files/MNIST/raw



5120it [00:00, 10791375.12it/s]         


Extracting ./tmp_files/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./tmp_files/MNIST/raw



In [6]:
learn = Learner(train_ds, setup_config)
learn.fit(10)
# a = iter(learn.data.train_dl)
# b, c = next(a)
# learn.learn.model(learn.learn.data.train_ds.x.shape[1], 10)
# next(learn.learn.model.parameters()).device
learn.device
learn.cbh.Monitor.history

100%|██████████| 187/187 [00:00<00:00, 1507.51it/s]


{'valid_acc': [0.582302807486631,
  0.6832804144385026,
  0.7347092245989305,
  0.7664396724598931,
  0.789221256684492,
  0.8066092914438503,
  0.8203900878533231,
  0.8318432486631016,
  0.8416239601901366,
  0.849966577540107],
 'valid_loss': [1.3281326249321515,
  0.7964778684358546,
  0.5955318681059036,
  0.49655550184415626,
  0.4314345890347333,
  0.3850051855499094,
  0.34832603759306635,
  0.31838473191554534,
  0.2946766298563085,
  0.2755472123782265],
 'train_loss': [1.932738372484843,
  1.002830522020658,
  0.646980444431305,
  0.5059469455281893,
  0.425909870048364,
  0.37137208676338196,
  0.3302723814745744,
  0.2977973664899667,
  0.2714618804951509,
  0.25005154422918957]}

In [8]:
learn.cbh.Recorder.batch_vals

{'train_loss': [2.313957452774048,
  2.296623706817627,
  2.3302738666534424,
  2.3264646530151367,
  2.306304454803467,
  2.297767400741577,
  2.310896873474121,
  2.323561906814575,
  2.3256783485412598,
  2.3004367351531982,
  2.3054358959198,
  2.306586503982544,
  2.3385674953460693,
  2.302555799484253,
  2.3164663314819336,
  2.307255983352661,
  2.307021141052246,
  2.2985289096832275,
  2.3405253887176514,
  2.3196523189544678,
  2.3219518661499023,
  2.3362185955047607,
  2.3290927410125732,
  2.315070867538452,
  2.334545850753784,
  2.294231653213501,
  2.294637680053711,
  2.318528175354004,
  2.2959792613983154,
  2.317267417907715,
  2.2955191135406494,
  2.309243679046631,
  2.3434252738952637,
  2.3018736839294434,
  2.3117709159851074,
  2.3312182426452637,
  2.3136019706726074,
  2.292043924331665,
  2.310678482055664,
  2.2907307147979736,
  2.2963149547576904,
  2.3164076805114746,
  2.336756944656372,
  2.322178602218628,
  2.34635591506958,
  2.29267954826355,
  

In [47]:
import types
class test_class():
    def __init__(self, a):
        self.attr = a
    
    def attr(self):
        pass

test = test_class(1)

getattr(test_class, "attr")

<function __main__.test_class.attr(self)>

In [35]:
%%timeit
test1.sum()

2.09 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [32]:
import numpy as np

In [16]:
class test1:
    def __init__(self, a):
        self.a = a
    def test_func(self):
        return self.a
a = test1(1)
setattr(a, "s", 123)
vars(a)

{'a': 1, 's': 123}

In [17]:
monitor = Monitor_Cb(["val_acc"])
early = EarlyStopping_Cb(monitor="val_loss")
recorder = Recorder_Cb("hi")
cbh = CallbackHandler([monitor, early, recorder])
cbh.Monitor_Cb.learn


NameError: name 'Monitor_Cb' is not defined

In [9]:
a = ["a"]
for i in a[:4]:
    print(i)

a
